In [1]:
import numpy as np
import pandas as pd
import re

In [2]:

def change_col_pos(col,pos,df):
    '''Change the position of a column in a dataframe: params(col - column name, 
    pos - new index position, df - dataframe)'''
    all_cols = df.columns.tolist()
    temp = all_cols.pop(all_cols.index(col))
    all_cols.insert(pos,col)
    
    df = df[all_cols]
    
    return df



In [3]:
#Se cambia la decodificación a latin-1 ya que con UTF-8 no se podía (aquí tengo duda)
df_original = pd.read_csv("C:\\Users\\josea_000\\Downloads\\attacks.csv", encoding = 'latin-1')
df = df_original.copy()
df.head(3)
print(df.shape)
print(df.columns)
print(len(df.columns))


(25723, 24)
Index(['Case Number', 'Date', 'Year', 'Type', 'Country', 'Area', 'Location',
       'Activity', 'Name', 'Sex ', 'Age', 'Injury', 'Fatal (Y/N)', 'Time',
       'Species ', 'Investigator or Source', 'pdf', 'href formula', 'href',
       'Case Number.1', 'Case Number.2', 'original order', 'Unnamed: 22',
       'Unnamed: 23'],
      dtype='object')
24


In [4]:
#Elimina las últimas dos columnas, son las que se encuentran casi vacías y, por lo tanto,
#son irrelevantes para el análisis de datos.
null_cols = df.isnull().sum()
null_cols
columnas_a_eliminar=list(null_cols[null_cols>25000].index)
df=df.drop(columnas_a_eliminar, axis=1)
df

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Fatal (Y/N),Time,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order
0,2018.06.25,25-jun-18,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,...,N,18h00,White shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.25,2018.06.25,6303.0
1,2018.06.18,18-jun-18,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,...,N,14h00 -15h00,NaN,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.18,2018.06.18,6302.0
2,2018.06.09,09-jun-18,2018.0,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,...,N,07h45,NaN,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.09,2018.06.09,6301.0
3,2018.06.08,08-jun-18,2018.0,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,...,N,NaN,2 m shark,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.08,2018.06.08,6300.0
4,2018.06.04,04-jun-18,2018.0,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,...,N,NaN,"Tiger shark, 3m",A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.04,2018.06.04,6299.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25718,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25719,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25720,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25721,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
#Crear las listas para eliminar aquellas filas en cuyo campo 'Case Number' estén vacías,
#contengan 0 o 'xx'.
filas_el=(df[df['Case Number'].isna()==True].index)
filas_el2=(df[df['Case Number']=='0'].index)
filas_el3=(df[df['Case Number']=='xx'].index)

In [6]:
df=df.drop(filas_el, axis=0)
df=df.drop(filas_el2, axis=0)
df=df.drop(filas_el3, axis=0)

In [7]:
df.shape

(6301, 22)

In [8]:
vacios=df.isnull().sum()
vacios
#Aunque se observa que 'Age','Time' y'Species' mantienen muchas filas de datos,
#se dejarán para el análisis. Si bien otros campos tienen filas con campos vacíos,
#pienso que es mínima la efectación en el análisis.

Case Number                  0
Date                         0
Year                         2
Type                         4
Country                     50
Area                       455
Location                   540
Activity                   544
Name                       210
Sex                        565
Age                       2830
Injury                      28
Fatal (Y/N)                539
Time                      3353
Species                   2837
Investigator or Source      17
pdf                          0
href formula                 1
href                         0
Case Number.1                0
Case Number.2                0
original order               0
dtype: int64

In [9]:
#Agregar una columna que tenga el mes, en caso de no tener incluye nan, sería bueno
#saber si existe un mes 'pico' en donde existen más ataques de tiburones y poder 
#relacionarlo con la estación del año, la temperatura promedio de ese mes, entre otras.

temp_lst = []
for row in df['Date']:
    temp_row = ''.join(re.findall('\-[a-zA-Z]{3}\-',row)).lower()
    temp_row = re.sub('\-','',temp_row)
        
        
    if temp_row == '':
        temp_row = np.nan

    temp_lst.append(temp_row)

df['month'] = temp_lst
print(df.shape)
df = change_col_pos('month',3,df)

(6301, 23)


In [10]:
df['month'].value_counts(dropna=False)

NaN       902
jul       621
aug       555
sep       519
jan       493
jun       475
apr       420
oct       417
dec       415
mar       379
nov       378
may       364
feb       357
marmar      2
augaug      1
sepsep      1
janjan      1
novnov      1
Name: month, dtype: int64

In [11]:
df.dropna(subset=['month'], inplace=True)
for row in df['month']:
    if len(row) > 3:
        df['month'].replace(row,row[:3], inplace=True)
    elif row == 'jut':
        df['month'].replace(row,'jun', inplace=True)

In [12]:
df['month'].value_counts(dropna=False)

jul    621
aug    556
sep    520
jan    494
jun    475
apr    420
oct    417
dec    415
mar    381
nov    379
may    364
feb    357
Name: month, dtype: int64

In [14]:
df

,Case Number,Date,Year,month,Type,Country,Area,Location,Activity,Name,...,Fatal (Y/N),Time,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order
0,2018.06.25,25-jun-18,2018.0,jun,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,...,N,18h00,White shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.25,2018.06.25,6303.0
1,2018.06.18,18-jun-18,2018.0,jun,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,...,N,14h00 -15h00,NaN,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.18,2018.06.18,6302.0
2,2018.06.09,09-jun-18,2018.0,jun,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,...,N,07h45,NaN,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.09,2018.06.09,6301.0
3,2018.06.08,08-jun-18,2018.0,jun,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,...,N,NaN,2 m shark,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.08,2018.06.08,6300.0
4,2018.06.04,04-jun-18,2018.0,jun,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,...,N,NaN,"Tiger shark, 3m",A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.04,2018.06.04,6299.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6231,ND.0082,"No date, Before 8-May-1965",0.0,may,Unprovoked,GREECE,Island of Volos,Eastern shore,Swimming,woman,...,Y,NaN,NaN,"Sydney Morning Herald, 5/8/1965",ND-0082-Volos-Greece.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0082,ND.0082,72.0
6245,ND.0060,Before 19-Jun-1959,0.0,jun,Unprovoked,USA,California,"Capistrano, Orange County",NaN,girl,...,N,NaN,"White shark, 1,900-lb","B. Walton, Sun (San Bernardino), 6/19/1959",ND-0060-Capistrano.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0060,ND.0060,58.0
6288,ND.0014,Before 17-Jul-1916,0.0,jul,Unprovoked,USA,North Carolina,Somewhere between Hatteras and Beaufort,Swimming,"""youthful male""",...,N,NaN,NaN,"C. Creswell, GSAF; Wilmington Star, 7/17/1916",ND-0014-pre1916-NorthCarolina.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0014,ND.0014,15.0
6289,ND.0013,No date (3 days after preceding incident) & pr...,0.0,jul,Unprovoked,SOUTH AFRICA,KwaZulu-Natal,Durban,Fishing,a native fisherman,...,Y,NaN,NaN,"Rural New Yorker, 7/19/1913",ND-0013-Durban-native-fisherman.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0013,ND.0013,14.0


In [18]:
df['Year']=df['Year'].astype('object')
df['original order']=df['original order'].astype('object')
df.dtypes


Case Number               object
Date                      object
Year                      object
month                     object
Type                      object
Country                   object
Area                      object
Location                  object
Activity                  object
Name                      object
Sex                       object
Age                       object
Injury                    object
Fatal (Y/N)               object
Time                      object
Species                   object
Investigator or Source    object
pdf                       object
href formula              object
href                      object
Case Number.1             object
Case Number.2             object
original order            object
dtype: object

In [ ]:
#Depende para qué se quiera la data, pero creo que ya se presenta de manera razonable.